# Data Engineering of the Google Store Analytics Dataset
This dataset is provided by the Kaggle competition.  
https://www.kaggle.com/c/ga-customer-revenue-prediction

A lot of columns have data in json format, which need to be split out into separate columns of data.

Also, want to add some further data based on the data/time of day.

In [6]:
import pandas as pd
import numpy as np

from datetime import datetime

### Import a Sample of the Training Dataset
Instead of working on the full training dataset, which has 903,653 rows, start by working on the top 10,000 rows of the full training dataset. 

In [7]:
df = pd.read_csv('data/train.csv', low_memory=False) #full training dataset

# df = pd.read_pickle('data/train_sample.pkl') #top 10,000 rows of the training dataset
print(df.shape)
print(df.columns)
df.head(3)

(903653, 12)
Index(['channelGrouping', 'date', 'device', 'fullVisitorId', 'geoNetwork',
       'sessionId', 'socialEngagementType', 'totals', 'trafficSource',
       'visitId', 'visitNumber', 'visitStartTime'],
      dtype='object')


,channelGrouping,date,device,fullVisitorId,geoNetwork,sessionId,socialEngagementType,totals,trafficSource,visitId,visitNumber,visitStartTime
0,Organic Search,20160902,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",1131660440785968503,"{""continent"": ""Asia"", ""subContinent"": ""Western...",1131660440785968503_1472830385,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1472830385,1,1472830385
1,Organic Search,20160902,"{""browser"": ""Firefox"", ""browserVersion"": ""not ...",377306020877927890,"{""continent"": ""Oceania"", ""subContinent"": ""Aust...",377306020877927890_1472880147,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1472880147,1,1472880147
2,Organic Search,20160902,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",3895546263509774583,"{""continent"": ""Europe"", ""subContinent"": ""South...",3895546263509774583_1472865386,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1472865386,1,1472865386


### Split out All Columns that Have JSON/Dictionary Info into Individual Columns
Some of the colums in the dataset have json/dictionary info - in order to be able to graph data/break out the variables, we want to separate each of this info to separate columns where each key of the dictionary is its own column data.

The columns that need to be split out are:
['device', 'geoNetwork', 'totals', 'trafficSource']

In [8]:
#print out shape of dataframe and names of all columns of the dataframe to see what columns are getting added as iterate through
print('original df shape: ', df.shape)
print('original df columns: ', df.columns)

#iterate through each of the column names that have to be split out into indiviual columns
for column in ['device', 'geoNetwork', 'totals', 'trafficSource']:

    #print out the name of the column you are processing in this for loop
    print('\n----------------------------------------\nsplitting out column: ', column)

    
    #some of the dictionaries have values of false/true instead of 'false'/'true'; python can't read the false/true
    #because is expecting False/True - instead of creating these boolean names, just convert them to text string
    #of 'false'/'true' before evaluating to dictionary
    df[column] = df[column].map(lambda x: x.replace("false", "'false'"))
    df[column] = df[column].map(lambda x: x.replace("true", "'true'"))
  

    #the JSON/dictionary columns are imported as string types - need to convert to python dictionaries using eval
    df[column] = df[column].map(lambda x: eval(x))

    
    #figure out all the possible unique keys for the dictionaries in this column - need this to make sure we
    #iterate through all of the dictionary data and store into different columns
    unique_column_keys = set()
    #iterate through each row of the column dictionaries
    for row_dictionary in list(df[column]):
        #iterate through each key of the row's dictionary and add to the unique column keys set (use add instead of append for sets)
        for row_key in list(row_dictionary.keys()):
            unique_column_keys.add(row_key)
    print('max length of column dictionaries is: ', len(unique_column_keys))
    print('unique column keys are: ', unique_column_keys)

    
    #use the set of unique_column_keys to add a new column for each of these keys
    for column_key in unique_column_keys:
        #create new column names of format:  originalcolumnname_columnkey, eg 'device_isMobile'
        df[f"{column}_{column_key}"] = df[column].map(lambda x: x.get(column_key, np.nan))
    
    print('updated df shape: ', df.shape)
    print('updated df columns: ', df.columns, '\n')

    
#show the final df head
df.head(3)

original df shape:  (903653, 12)
original df columns:  Index(['channelGrouping', 'date', 'device', 'fullVisitorId', 'geoNetwork',
       'sessionId', 'socialEngagementType', 'totals', 'trafficSource',
       'visitId', 'visitNumber', 'visitStartTime'],
      dtype='object')

----------------------------------------
splitting out column:  device
max length of column dictionaries is:  16
unique column keys are:  {'browserSize', 'operatingSystem', 'screenResolution', 'browser', 'browserVersion', 'flashVersion', 'mobileDeviceBranding', 'operatingSystemVersion', 'mobileDeviceModel', 'language', 'mobileDeviceInfo', 'screenColors', 'isMobile', 'mobileInputSelector', 'mobileDeviceMarketingName', 'deviceCategory'}
updated df shape:  (903653, 28)
updated df columns:  Index(['channelGrouping', 'date', 'device', 'fullVisitorId', 'geoNetwork',
       'sessionId', 'socialEngagementType', 'totals', 'trafficSource',
       'visitId', 'visitNumber', 'visitStartTime', 'device_browserSize',
       'devic

,channelGrouping,date,device,fullVisitorId,geoNetwork,sessionId,socialEngagementType,totals,trafficSource,visitId,...,totals_hits,trafficSource_campaign,trafficSource_referralPath,trafficSource_campaignCode,trafficSource_adwordsClickInfo,trafficSource_keyword,trafficSource_medium,trafficSource_isTrueDirect,trafficSource_adContent,trafficSource_source
0,Organic Search,20160902,"{'browser': 'Chrome', 'browserVersion': 'not a...",1131660440785968503,"{'continent': 'Asia', 'subContinent': 'Western...",1131660440785968503_1472830385,Not Socially Engaged,"{'visits': '1', 'hits': '1', 'pageviews': '1',...","{'campaign': '(not set)', 'source': 'google', ...",1472830385,...,1,(not set),NaN,NaN,{'criteriaParameters': 'not available in demo ...,(not provided),organic,NaN,NaN,google
1,Organic Search,20160902,"{'browser': 'Firefox', 'browserVersion': 'not ...",377306020877927890,"{'continent': 'Oceania', 'subContinent': 'Aust...",377306020877927890_1472880147,Not Socially Engaged,"{'visits': '1', 'hits': '1', 'pageviews': '1',...","{'campaign': '(not set)', 'source': 'google', ...",1472880147,...,1,(not set),NaN,NaN,{'criteriaParameters': 'not available in demo ...,(not provided),organic,NaN,NaN,google
2,Organic Search,20160902,"{'browser': 'Chrome', 'browserVersion': 'not a...",3895546263509774583,"{'continent': 'Europe', 'subContinent': 'South...",3895546263509774583_1472865386,Not Socially Engaged,"{'visits': '1', 'hits': '1', 'pageviews': '1',...","{'campaign': '(not set)', 'source': 'google', ...",1472865386,...,1,(not set),NaN,NaN,{'criteriaParameters': 'not available in demo ...,(not provided),organic,NaN,NaN,google


### Exploration - Determine the Value Counts of each Column
To get a better grasp of the data that we have, look at value counts of each column.  Some will have lots of different values (like fullVisitorId, which will be unique to every individual).  Others will have binary info.

Cycle through and print the top 10 value_counts for each column.

In [9]:
#iterate through every single column in the dataset (that is not a dictionary, can't do value_counts on a dictionary)
for column in df.columns:

    #print out the name of the column you are processing in this for loop
    print('\n----------------------------------------\nCOLUMN: ', column)
    
    #if column has dictionaries, it can't do value_counts so flag this in output
    if type(df[column][0]) == dict:
        print('column contains dictionaries')
        print('example dictionary from 0th row: \n', df[column][0])
    
    else:
        #print out the length of the value_counts for the column
        print('# of Unique Values in Column: ', len(df[column].value_counts()))

        #print out the top ten value_counts for the column
        #normalize=True provides relative frequency of the value (instead of just count), so 0.473 is 47.3%
        print('Top 10 Value Counts for Column: \n', df[column].value_counts(normalize=True).head(10))



----------------------------------------
COLUMN:  channelGrouping
# of Unique Values in Column:  8
Top 10 Value Counts for Column: 
 Organic Search    0.422243
Social            0.250225
Direct            0.158275
Referral          0.116016
Paid Search       0.028026
Affiliates        0.018152
Display           0.006930
(Other)           0.000133
Name: channelGrouping, dtype: float64

----------------------------------------
COLUMN:  date
# of Unique Values in Column:  366
Top 10 Value Counts for Column: 
 20161128    0.005320
20161115    0.005185
20161114    0.004942
20161130    0.004908
20161026    0.004841
20161129    0.004799
20161116    0.004796
20161004    0.004783
20161205    0.004720
20170426    0.004674
Name: date, dtype: float64

----------------------------------------
COLUMN:  device
column contains dictionaries
example dictionary from 0th row: 
 {'browser': 'Chrome', 'browserVersion': 'not available in demo dataset', 'browserSize': 'not available in demo dataset', 'operat

# of Unique Values in Column:  1
Top 10 Value Counts for Column: 
 not available in demo dataset    1.0
Name: device_mobileDeviceMarketingName, dtype: float64

----------------------------------------
COLUMN:  device_deviceCategory
# of Unique Values in Column:  3
Top 10 Value Counts for Column: 
 desktop    0.735325
mobile     0.230979
tablet     0.033695
Name: device_deviceCategory, dtype: float64

----------------------------------------
COLUMN:  geoNetwork_networkLocation
# of Unique Values in Column:  1
Top 10 Value Counts for Column: 
 not available in demo dataset    1.0
Name: geoNetwork_networkLocation, dtype: float64

----------------------------------------
COLUMN:  geoNetwork_longitude
# of Unique Values in Column:  1
Top 10 Value Counts for Column: 
 not available in demo dataset    1.0
Name: geoNetwork_longitude, dtype: float64

----------------------------------------
COLUMN:  geoNetwork_subContinent
# of Unique Values in Column:  23
Top 10 Value Counts for Column: 
 Nort

# of Unique Values in Column:  7
Top 10 Value Counts for Column: 
 organic      0.422243
referral     0.366241
(none)       0.158275
cpc          0.028026
affiliate    0.018152
cpm          0.006930
(not set)    0.000133
Name: trafficSource_medium, dtype: float64

----------------------------------------
COLUMN:  trafficSource_isTrueDirect
# of Unique Values in Column:  1
Top 10 Value Counts for Column: 
 true    1.0
Name: trafficSource_isTrueDirect, dtype: float64

----------------------------------------
COLUMN:  trafficSource_adContent
# of Unique Values in Column:  44
Top 10 Value Counts for Column: 
 Google Merchandise Collection    0.467933
Google Online Store              0.113740
Display Ad created 3/11/14       0.088343
Full auto ad IMAGE ONLY          0.075096
Ad from 12/13/16                 0.055728
Ad from 11/3/16                  0.044674
Display Ad created 3/11/15       0.035812
{KeyWord:Google Brand Items}     0.022931
{KeyWord:Google Merchandise}     0.014160
Ad from 1

### Save Extracted Data to Pickles

In [10]:
# df.to_pickle('data/train_sample_split_v1.pkl')

In [11]:
# df.columns

In [12]:
#drop original columns that had dictionary/jsons
df_dropped = df.drop(['device', 'geoNetwork', 'totals', 'trafficSource'], axis=1)
print(df_dropped.shape)
df_dropped.head(3)

(903653, 50)


,channelGrouping,date,fullVisitorId,sessionId,socialEngagementType,visitId,visitNumber,visitStartTime,device_browserSize,device_operatingSystem,...,totals_hits,trafficSource_campaign,trafficSource_referralPath,trafficSource_campaignCode,trafficSource_adwordsClickInfo,trafficSource_keyword,trafficSource_medium,trafficSource_isTrueDirect,trafficSource_adContent,trafficSource_source
0,Organic Search,20160902,1131660440785968503,1131660440785968503_1472830385,Not Socially Engaged,1472830385,1,1472830385,not available in demo dataset,Windows,...,1,(not set),NaN,NaN,{'criteriaParameters': 'not available in demo ...,(not provided),organic,NaN,NaN,google
1,Organic Search,20160902,377306020877927890,377306020877927890_1472880147,Not Socially Engaged,1472880147,1,1472880147,not available in demo dataset,Macintosh,...,1,(not set),NaN,NaN,{'criteriaParameters': 'not available in demo ...,(not provided),organic,NaN,NaN,google
2,Organic Search,20160902,3895546263509774583,3895546263509774583_1472865386,Not Socially Engaged,1472865386,1,1472865386,not available in demo dataset,Windows,...,1,(not set),NaN,NaN,{'criteriaParameters': 'not available in demo ...,(not provided),organic,NaN,NaN,google


In [6]:
#save the dataframe to pickle (use the df that has dropped the original dict/json columns since it cuts size almost in half)
df_dropped.to_pickle('data/train_full_data_split.pkl')

In [14]:
# #save the dataframe to csv (use the df that has dropped the original dict/json columns since it cuts size almost in half)
# df_dropped.to_csv('data/train_full_data_split.csv')

### Add in Additional Date Info (day of week, holiday, etc.)
1. Start with the date provided, which has just imported as a long int value, for example: 20170126 is 2017-01-26, so yearmonthday format.
2. Convert int value into a datetime value.
3. Then use the datetime value to add a column that defines day of week
4. Also add a column that separates out month of year from date.
5. Lastly, try to add a column to determine whether the date is a holiday or not.

In [8]:
df.date[0]

20160902

In [4]:
df2 = pd.read_pickle('data/train_full_data_split.pkl')

In [5]:
df2.to_csv('test_full.csv')

UnicodeEncodeError: 'utf-8' codec can't encode characters in position 726-727: surrogates not allowed